# Feature selection exercises

How do we measure the performance of a feature selection algorithm? This is a difficult question and we should consider a few aspects of the answer:
- computation time
- size of the resulting feature set (the minimal description length principle)
- feature set stability (whether the set of selected features changes a lot when we slightly change our data)
- performance of prediction models (the ability to identify valid dependencies in the data)

There are several popular methods of testing the FS performance. 
1. The best way - compare the resulting feature sets to the ground truth. Unfortunately, in practice, this is possible only for synthetic data or relatively simple data that can be analyzed by experts.
2. If we cannot directly compare with the ground truth, then we can at least estimate how often a given algorithm makes obvious errors. One way of doing that is to artificially add a set of random probes to the data before we start our analysis and check what portion of the selected features comes from this set.
3. We may draw several bootstrap samples of data a empirically check the stability of FS.
4. We may assess the FS algorithm indirectly by estimating the performance of a prediction algorithm that uses the selected features. **This method requires caution**. We must not use the same data for FS and the estimation!

## Gollub's experiment


In [2]:
library(class)
nRows = 100
nCols = 10000

# Let's generate some random data...
dataTab = as.data.frame(matrix(runif(nRows*nCols, -1, 1), nRows, nCols))
# and a random decision vector
decisionClasses = sample(c(0,1), nRows, replace = TRUE)
# we may rank the features using any method, e.g. correlation filter, and select the top 5
corScores = abs(cor(dataTab, decisionClasses))
selectedAttrs = order(corScores, decreasing = TRUE)[1:5]

# now, we can measure performance of a prediction model
predsAll = knn.cv(dataTab[selectedAttrs], decisionClasses, k = 3)
cat('The accuracy of k-NN is: ', mean(as.character(decisionClasses) == as.character(predsAll)), '\n')


The accuracy of k-NN is:  0.72 


What can you say about the above results? As the first exercise, please repeat the experiment using a proper methodology. Compare the outcomes. What are your conclusions?

## An experiment with synthetic data

Let's define another synthetic data but this time, let's make the classification vector dependent on the first 10 attributes (see the examples below). Try to assess the performance of the discussed feature subset selection methods with regard to the difficulty (complexity) of the decision attribute.


In [8]:
nRows = 1000
nCols = 100

# Let's generate some random data...
dataTab = as.data.frame(matrix(runif(nRows*nCols, -1, 1), nRows, nCols))

decisions1 = apply(dataTab, 1, function(x) as.integer(sum(x[1:10]) > 0))
# introduce some noise
tmp = sample(nRows, 150)
decisions1[tmp] = abs(decisions1[tmp] - 1)
table(decisions1)
                   
decisions2 = apply(dataTab, 1, 
                   function(x) as.integer((x[1] > 0 & x[2] > 0 & x[3] < 0) | (x[3] > 0 & sum(x[4:10]) < 0)))
tmp = sample(nRows, 150)
decisions2[tmp] = abs(decisions2[tmp] - 1)
table(decisions2)
                   
decisions3 = apply(dataTab, 1, 
                   function(x) as.integer((sum(x[1:4]) < 0.5 & sum(x[1:4]) > -0.5 & 
                                           sum(x[5:8]) < 0.5 & sum(x[5:8]) > -0.5) |
                                          (x[9] < 0.5 & x[9] > -0.5 & abs(x[10] * x[1]) < 0.25)))
tmp = sample(nRows, 150)
decisions3[tmp] = abs(decisions3[tmp] - 1)
table(decisions3)

cor(cbind(decisions1, decisions2, decisions3))


decisions1
  0   1 
513 487 

decisions2
  0   1 
603 397 

decisions3
  0   1 
569 431 

,decisions1,decisions2,decisions3
decisions1,1.00000000,-0.09134539,0.01253614
decisions2,-0.09134539,1.00000000,-0.05409469
decisions3,0.01253614,-0.05409469,1.00000000


## A real-world example - a microarray data set

In this task, you will have to face the curse of dimensionality. Your task is to analyze the *acuteLymphoblasticLeukemia* data. Use some of the techniques we discussed (choose a few) to select and evaluate sets of genes that allow classifying cases of leukemia into subtypes.

Choose your FS algorithms wisely as not all of the discussed methods are computationally feasible in this task.


In [4]:
library(data.table)

leukemiaDT = data.table::fread('acuteLymphoblasticLeukemia.data', header = TRUE)
dim(leukemiaDT)
head(colnames(leukemiaDT))
tail(colnames(leukemiaDT))

table(leukemiaDT$Decision)


Read 190 rows and 22278 (of 22278) columns from 0.048 GB file in 00:00:06


[1]   190 22278

[1] "Decision"       "AFFX-BioB-3_at" "AFFX-BioB-5_at" "AFFX-BioB-M_at"
[5] "AFFX-BioC-3_at" "AFFX-BioC-5_at"

[1] "222379_at"   "222380_s_at" "222381_at"   "222382_x_at" "222383_s_at"
[6] "222384_at"


 Precursor-B  ALL, subtype: Hyperdiploid 
                                      44 
Precursor-B ALL, subtype: E2A-rearranged 
                                      13 
         Precursor-B ALL, subtype: other 
                                      53 
      Precursor-B ALL, subtype: TEL-AML1 
                                      44 
                                   T-ALL 
                                      36 